<a href="https://colab.research.google.com/github/evegat/usm2025-1-dip-ia-edu-v2-evegat/blob/main/Proyecto/07prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hito 3: Prueba de modelo con ejemplos personalizados

En este notebook se realizan predicciones individuales utilizando el modelo Random Forest entrenado previamente. Se construyen tres ejemplos realistas con distintas combinaciones de características para evaluar cómo responde el modelo. Esto permite verificar su utilidad práctica y reflexionar sobre sus posibles aplicaciones en contextos reales.


In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar dataset desde GitHub
url = 'https://github.com/evegat/usm2025-1-dip-ia-edu-v2-evegat/raw/main/Proyecto/01database.csv.gz'
df = pd.read_csv(url, compression='gzip')

# Preprocesamiento
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

df['region_sede'] = df['region_sede'].astype(str)
df['region_sede_cod'] = LabelEncoder().fit_transform(df['region_sede'])

# Normalizar columnas numéricas
columnas_numericas = ['valor_matricula', 'valor_arancel']
df[columnas_numericas] = MinMaxScaler().fit_transform(df[columnas_numericas])

# Crear variable objetivo
df['es_pedagogia'] = df['nomb_carrera'].str.contains('PEDAGOG', case=False, na=False).astype(int)

# Variables predictoras
X = df[['gen_alu', 'rango_edad', 'region_sede_cod', 'tipo_inst_1', 'modalidad',
        'jornada', 'nivel_carrera_1', 'forma_ingreso', 'acreditada_carr',
        'valor_matricula', 'valor_arancel']].copy()
y = df['es_pedagogia']

# Codificar columnas categóricas
for col in X.select_dtypes(include='object').columns:
    X.loc[:, col] = LabelEncoder().fit_transform(X[col].astype(str))

# Dividir dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo final
modelo = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
modelo.fit(X_train, y_train)


RandomForestClassifier(max_depth=10, random_state=42)

## Ejecución de ejemplos personalizados

A continuación, se simulan tres perfiles con características distintas para predecir si una persona estudiará o no pedagogía. Las variables han sido codificadas como se entrenó el modelo.


In [2]:
# Crear tres ejemplos con valores consistentes con el dataset
ejemplos = pd.DataFrame({
    'gen_alu': [0, 1, 1],                 # 0: mujer, 1: hombre
    'rango_edad': [1, 3, 2],              # codificación previa
    'region_sede_cod': [12, 4, 7],        # Metropolitana, Biobío, O'Higgins
    'tipo_inst_1': [0, 1, 0],             # CFT, IP, CFT
    'modalidad': [0, 1, 0],               # Presencial, Online, Presencial
    'jornada': [0, 1, 0],                 # Diurna, Vespertina, Diurna
    'nivel_carrera_1': [0, 1, 2],         # Técnica, Profesional, Otro
    'forma_ingreso': [0, 2, 3],           # Regular, PACE, Especial
    'acreditada_carr': [1, 0, 1],         # Acreditada o no
    'valor_matricula': [0.3, 0.1, 0.6],   # Valores ya normalizados
    'valor_arancel': [0.4, 0.2, 0.8]
})

print("Ejemplos de entrada:")
print(ejemplos)


Ejemplos de entrada:
   gen_alu  rango_edad  region_sede_cod  tipo_inst_1  modalidad  jornada  \
0        0           1               12            0          0        0   
1        1           3                4            1          1        1   
2        1           2                7            0          0        0   

   nivel_carrera_1  forma_ingreso  acreditada_carr  valor_matricula  \
0                0              0                1              0.3   
1                1              2                0              0.1   
2                2              3                1              0.6   

   valor_arancel  
0            0.4  
1            0.2  
2            0.8  


In [4]:
# Generar predicciones
predicciones = modelo.predict(ejemplos)

# Mostrar resultados
print("\nResultados del modelo:")
for i, pred in enumerate(predicciones):
    decision = "Estudia Pedagogía" if pred == 1 else "No estudia Pedagogía"
    print(f"Ejemplo {i+1}: {decision}")




Resultados del modelo:
Ejemplo 1: No estudia Pedagogía
Ejemplo 2: No estudia Pedagogía
Ejemplo 3: No estudia Pedagogía


## Conclusiones

Las predicciones del modelo permiten simular decisiones educativas para distintos perfiles de estudiantes. En los tres ejemplos construidos, el modelo consideró que ninguno ingresaría a pedagogía. Este tipo de resultados puede ayudar a analizar patrones institucionales o sesgos en la admisión o auto-selección según variables como la región, tipo de institución o forma de ingreso.

Este enfoque puede ampliarse para exploración masiva de escenarios, diseño de políticas o focalización de intervenciones que promuevan el acceso a pedagogía, considerando las brechas estructurales detectadas por el modelo.
